<a href="https://colab.research.google.com/github/chenxr35/AI6127-Assignment2/blob/main/Assignment_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%matplotlib inline

In [2]:
!pip install torchmetrics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 841.5/841.5 kB 5.5 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-many

Translation with a Sequence to Sequence Network
*************************************************************

::

    [KEY: > input, = target, < output]

    > il est en train de peindre un tableau .
    = he is painting a picture .
    < he is painting a picture .

    > pourquoi ne pas essayer ce vin delicieux ?
    = why not try that delicious wine ?
    < why not try that delicious wine ?

    > elle n est pas poete mais romanciere .
    = she is not a poet but a novelist .
    < she not not a poet but a novelist .

    > vous etes trop maigre .
    = you re too skinny .
    < you re all alone .

... to varying degrees of success.

This is made possible by the simple but powerful idea of the `sequence
to sequence network <http://arxiv.org/abs/1409.3215>`__, in which two
recurrent neural networks work together to transform one sequence to
another. An encoder network condenses an input sequence into a vector,
and a decoder network unfolds that vector into a new sequence.

.. figure:: /_static/img/seq-seq-images/seq2seq.png
   :alt:

To improve upon this model we'll use an `attention
mechanism <https://arxiv.org/abs/1409.0473>`__, which lets the decoder
learn to focus over a specific range of the input sequence.

**Recommended Reading:**

I assume you have at least installed PyTorch, know Python, and
understand Tensors:

-  http://pytorch.org/ For installation instructions
-  :doc:`/beginner/deep_learning_60min_blitz` to get started with PyTorch in general
-  :doc:`/beginner/pytorch_with_examples` for a wide and deep overview
-  :doc:`/beginner/former_torchies_tutorial` if you are former Lua Torch user


It would also be useful to know about Sequence to Sequence networks and
how they work:

-  `Learning Phrase Representations using RNN Encoder-Decoder for
   Statistical Machine Translation <http://arxiv.org/abs/1406.1078>`__
-  `Sequence to Sequence Learning with Neural
   Networks <http://arxiv.org/abs/1409.3215>`__
-  `Neural Machine Translation by Jointly Learning to Align and
   Translate <https://arxiv.org/abs/1409.0473>`__
-  `A Neural Conversational Model <http://arxiv.org/abs/1506.05869>`__

You will also find the previous tutorials on
:doc:`/intermediate/char_rnn_classification_tutorial`
and :doc:`/intermediate/char_rnn_generation_tutorial`
helpful as those concepts are very similar to the Encoder and Decoder
models, respectively.

And for more, read the papers that introduced these topics:

-  `Learning Phrase Representations using RNN Encoder-Decoder for
   Statistical Machine Translation <http://arxiv.org/abs/1406.1078>`__
-  `Sequence to Sequence Learning with Neural
   Networks <http://arxiv.org/abs/1409.3215>`__
-  `Neural Machine Translation by Jointly Learning to Align and
   Translate <https://arxiv.org/abs/1409.0473>`__
-  `A Neural Conversational Model <http://arxiv.org/abs/1506.05869>`__


**Requirements**



Loading data files
==================

The data for this project is a set of many thousands of English to
French translation pairs.

`This question on Open Data Stack
Exchange <http://opendata.stackexchange.com/questions/3888/dataset-of-sentences-translated-into-many-languages>`__
pointed me to the open translation site http://tatoeba.org/ which has
downloads available at http://tatoeba.org/eng/downloads - and better
yet, someone did the extra work of splitting language pairs into
individual text files here: http://www.manythings.org/anki/

The English to French pairs are too big to include in the repo, so
download to ``data/eng-fra.txt`` before continuing. The file is a tab
separated list of translation pairs:

::

    I am cold.    J'ai froid.

.. Note::
   Download the data from
   `here <https://download.pytorch.org/tutorial/data.zip>`_
   and extract it to the current directory.



In [3]:
!wget http://www.manythings.org/anki/fra-eng.zip
!unzip -o fra-eng.zip
!mkdir data
!mv fra.txt data/eng-fra.txt

--2024-04-13 08:21:38--  http://www.manythings.org/anki/fra-eng.zip
Resolving www.manythings.org (www.manythings.org)... 173.254.30.110
Connecting to www.manythings.org (www.manythings.org)|173.254.30.110|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7943074 (7.6M) [application/zip]
Saving to: ‘fra-eng.zip’

fra-eng.zip         100%[===================>]   7.57M  35.8MB/s    in 0.2s    

2024-04-13 08:21:39 (35.8 MB/s) - ‘fra-eng.zip’ saved [7943074/7943074]

Archive:  fra-eng.zip
  inflating: _about.txt              
  inflating: fra.txt                 


In [4]:
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import string
import re
import random
SOS_token = 0
EOS_token = 1


class Lang:
    def __init__(self, name):
        self.name = name
        self.word2index = {}
        self.word2count = {}
        self.index2word = {0: "SOS", 1: "EOS"}
        self.n_words = 2  # Count SOS and EOS

    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1
import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

The files are all in Unicode, to simplify we will turn Unicode
characters to ASCII, make everything lowercase, and trim most
punctuation.




In [5]:
# Turn a Unicode string to plain ASCII, thanks to
# http://stackoverflow.com/a/518232/2809427
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )

# Lowercase, trim, and remove non-letter characters


def normalizeString(s):
    s = unicodeToAscii(s.lower().strip())
    s = re.sub(r"([.!?])", r" \1", s)
    s = re.sub(r"[^a-zA-Z.!?]+", r" ", s)
    return s

To read the data file we will split the file into lines, and then split
lines into pairs. The files are all English → Other Language, so if we
want to translate from Other Language → English I added the ``reverse``
flag to reverse the pairs.




In [6]:
def readLangs(lang1, lang2, reverse=False):
    print("Reading lines...")

    # Read the file and split into lines
    lines = open('data/%s-%s.txt' % (lang1, lang2), encoding='utf-8').\
        read().strip().split('\n')

    # Split every line into pairs and normalize
    pairs = [[normalizeString(s) for s in l.split('\t')[:2]] for l in lines]

    # Reverse pairs, make Lang instances
    if reverse:
        pairs = [list(reversed(p)) for p in pairs]
        input_lang = Lang(lang2)
        output_lang = Lang(lang1)
    else:
        input_lang = Lang(lang1)
        output_lang = Lang(lang2)

    return input_lang, output_lang, pairs

Since there are a *lot* of example sentences and we want to train
something quickly, we'll trim the data set to only relatively short and
simple sentences. Here the maximum length is 10 words (that includes
ending punctuation) and we're filtering to sentences that translate to
the form "I am" or "He is" etc. (accounting for apostrophes replaced
earlier).




In [7]:
MAX_LENGTH = 15

eng_prefixes = (
    "i am", "i m",
    "he is", "he s",
    "she is", "she s",
    "you are", "you re",
    "we are", "we re",
    "they are", "they re"
)


def filterPair(p):
    return len(p[0].split(' ')) < MAX_LENGTH and \
        len(p[1].split(' ')) < MAX_LENGTH and \
        p[1].startswith(eng_prefixes)


def filterPairs(pairs):
    return [pair for pair in pairs if filterPair(pair)]

The full process for preparing the data is:

-  Read text file and split into lines, split lines into pairs
-  Normalize text, filter by length and content
-  Make word lists from sentences in pairs




In [8]:
def prepareData(lang1, lang2, reverse=False):
    input_lang, output_lang, pairs = readLangs(lang1, lang2, reverse)
    print("Read %s sentence pairs" % len(pairs))
    pairs = filterPairs(pairs)
    print("Trimmed to %s sentence pairs" % len(pairs))
    print("Counting words...")
    for pair in pairs:
        input_lang.addSentence(pair[0])
        output_lang.addSentence(pair[1])
    print("Counted words:")
    print(input_lang.name, input_lang.n_words)
    print(output_lang.name, output_lang.n_words)
    return input_lang, output_lang, pairs


input_lang, output_lang, pairs = prepareData('eng', 'fra', True)
#print(random.choice(pairs))

Reading lines...
Read 232736 sentence pairs
Trimmed to 22907 sentence pairs
Counting words...
Counted words:
fra 7019
eng 4638


In [9]:
from sklearn.model_selection import train_test_split

In [10]:
X = [i[0] for i in pairs]
y = [i[1] for i in pairs]

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

In [12]:
train_pairs = list(zip(X_train,y_train))
test_pairs = list(zip(X_test,y_test))

The Seq2Seq Model
=================

A Recurrent Neural Network, or RNN, is a network that operates on a
sequence and uses its own output as input for subsequent steps.

A `Sequence to Sequence network <http://arxiv.org/abs/1409.3215>`__, or
seq2seq network, or `Encoder Decoder
network <https://arxiv.org/pdf/1406.1078v3.pdf>`__, is a model
consisting of two RNNs called the encoder and decoder. The encoder reads
an input sequence and outputs a single vector, and the decoder reads
that vector to produce an output sequence.

Unlike sequence prediction with a single RNN, where every input
corresponds to an output, the seq2seq model frees us from sequence
length and order, which makes it ideal for translation between two
languages.

Consider the sentence "Je ne suis pas le chat noir" → "I am not the
black cat". Most of the words in the input sentence have a direct
translation in the output sentence, but are in slightly different
orders, e.g. "chat noir" and "black cat". Because of the "ne/pas"
construction there is also one more word in the input sentence. It would
be difficult to produce a correct translation directly from the sequence
of input words.

With a seq2seq model the encoder creates a single vector which, in the
ideal case, encodes the "meaning" of the input sequence into a single
vector — a single point in some N dimensional space of sentences.




The Encoder
-----------

The encoder of a seq2seq network is a RNN that outputs some value for
every word from the input sentence. For every input word the encoder
outputs a vector and a hidden state, and uses the hidden state for the
next input word.






In [13]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)

    def forward(self, input, hidden):
        embedded = self.embedding(input).view(1, 1, -1)
        output = embedded
        output, hidden = self.gru(output, hidden)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

### LSTM Encoder

In [65]:
class EncoderLSTM(nn.Module):
    def __init__(self, input_size, hidden_size, bidirectional=False):
        super(EncoderLSTM, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(input_size, hidden_size)
        self.lstm = nn.LSTM(hidden_size, hidden_size, bidirectional=bidirectional)

    def forward(self, input, states):
        embedded = self.embedding(input).view(1, 1, -1)
        output = embedded
        hidden, cell = states[0], states[1]
        output, (hidden, cell) = self.lstm(output, (hidden, cell))
        return output, (hidden, cell)

    def initHidden(self):
        if self.lstm.bidirectional:
            return (torch.zeros(2, 1, self.hidden_size, device=device), torch.zeros(2, 1, self.hidden_size, device=device))
        return (torch.zeros(1, 1, self.hidden_size, device=device), torch.zeros(1, 1, self.hidden_size, device=device))

### Transformer Encoder

In [13]:
from torch import Tensor

class PositionalEncoding(nn.Module):

    def __init__(self, d_model: int, dropout: float = 0.1, max_len: int = MAX_LENGTH):
        super().__init__()
        self.dropout = nn.Dropout(p=dropout)

        position = torch.arange(max_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2) * (-math.log(10000.0) / d_model))
        pe = torch.zeros(max_len, 1, d_model)
        pe[:, 0, 0::2] = torch.sin(position * div_term)
        pe[:, 0, 1::2] = torch.cos(position * div_term)
        self.register_buffer('pe', pe)

    def forward(self, x: Tensor) -> Tensor:
        """
        Arguments:
            x: Tensor, shape ``[seq_len, batch_size, embedding_dim]``
        """
        x = x + self.pe[:x.size(0)]
        return self.dropout(x)

In [14]:
class EncoderTransformer(nn.Module):
    def __init__(self, input_size, hidden_size, nhead, num_layers):
        super(EncoderTransformer, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(input_size, hidden_size)
        self.pos_encoder = PositionalEncoding(hidden_size)
        encoder_layers = nn.TransformerEncoderLayer(hidden_size, nhead, hidden_size)
        self.transformer_encoder = nn.TransformerEncoder(encoder_layers, num_layers)

        self.init_weights()

    def init_weights(self):
        initrange = 0.1
        self.embedding.weight.data.uniform_(-initrange, initrange)

    def forward(self, input):
        embedded = self.embedding(input) * math.sqrt(self.hidden_size)
        embedded = self.pos_encoder(embedded)
        output = self.transformer_encoder(embedded)
        return output

The Decoder (Your assignment)
-----------

The decoder is another RNN that takes the encoder output vector(s) and
outputs a sequence of words to create the translation.




Simple Decoder
^^^^^^^^^^^^^^

In the simplest seq2seq decoder we use only last output of the encoder.
This last output is sometimes called the *context vector* as it encodes
context from the entire sequence. This context vector is used as the
initial hidden state of the decoder.

At every step of decoding, the decoder is given an input token and
hidden state. The initial input token is the start-of-string ``<SOS>``
token, and the first hidden state is the context vector (the encoder's
last hidden state).


In [15]:
class Decoder(nn.Module):
    def __init__(self, hidden_size, output_size):
        super(Decoder, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(output_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden):
        output = self.embedding(input).view(1, 1, -1)
        output = F.relu(output)
        # (hidden, cell) -> hidden
        if len(hidden) == 2:
            hidden = hidden[0]
        # concatenate hidden states from Bi-LSTM encoder
        if hidden.shape[0] == 2:
            hidden = torch.cat((hidden[0], hidden[1]), dim = 1)
            hidden = hidden.view(1, 1, -1)
        output, hidden = self.gru(output, hidden)
        output = self.softmax(self.out(output[0]))
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

### LSTM Decoder

In [51]:
class DecoderLSTM(nn.Module):
    def __init__(self, hidden_size, output_size):
        super(DecoderLSTM, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(output_size, hidden_size)
        self.lstm = nn.LSTM(hidden_size, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, states):
        output = self.embedding(input).view(1, 1, -1)
        output = F.relu(output)
        hidden, cell = states[0], states[1]
        output, (hidden, cell) = self.lstm(output, (hidden, cell))
        output = self.softmax(self.out(output[0]))
        return output, (hidden, cell)

    def initHidden(self):
        return (torch.zeros(1, 1, self.hidden_size, device=device), torch.zeros(1, 1, self.hidden_size, device=device))

### Decoder with Attention Mechanism

In [34]:
class BahdanauAttention(nn.Module):
    def __init__(self, hidden_size):
        super(BahdanauAttention, self).__init__()
        self.Wa = nn.Linear(hidden_size, hidden_size)
        self.Ua = nn.Linear(hidden_size, hidden_size)
        self.Va = nn.Linear(hidden_size, 1)

    def forward(self, query, keys):
        scores = self.Va(torch.tanh(self.Wa(query) + self.Ua(keys)))
        scores = scores.squeeze(2).unsqueeze(1)

        weights = F.softmax(scores, dim=-1)
        context = torch.bmm(weights, keys)

        return context, weights

In [28]:
class AttentionDecoder(nn.Module):
    def __init__(self, hidden_size, output_size):
        super(AttentionDecoder, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(output_size, hidden_size)
        self.attention = BahdanauAttention(hidden_size)
        self.gru = nn.GRU(2 * hidden_size, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden, encoder_outputs):
        embedded = self.embedding(input).view(1, 1, -1)

        query = hidden.permute(1, 0, 2)
        context, attn_weights = self.attention(query, encoder_outputs)
        input_gru = torch.cat((embedded, context), dim=2)

        output, hidden = self.gru(input_gru, hidden)
        output = self.softmax(self.out(output[0]))

        return output, hidden, attn_weights

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

<div class="alert alert-info"><h4>Note</h4><p>There are other forms of attention that work around the length
  limitation by using a relative position approach. Read about "local
  attention" in `Effective Approaches to Attention-based Neural Machine
  Translation <https://arxiv.org/abs/1508.04025>`__.</p></div>

Training
========

Preparing Training Data
-----------------------

To train, for each pair we will need an input tensor (indexes of the
words in the input sentence) and target tensor (indexes of the words in
the target sentence). While creating these vectors we will append the
EOS token to both sequences.




In [16]:
def indexesFromSentence(lang, sentence):
    return [lang.word2index[word] for word in sentence.split(' ')]


def tensorFromSentence(lang, sentence):
    indexes = indexesFromSentence(lang, sentence)
    indexes.append(EOS_token)
    return torch.tensor(indexes, dtype=torch.long, device=device).view(-1, 1)


def tensorsFromPair(pair):
    input_tensor = tensorFromSentence(input_lang, pair[0])
    target_tensor = tensorFromSentence(output_lang, pair[1])
    return (input_tensor, target_tensor)

Training the Model
------------------

To train we run the input sentence through the encoder, and keep track
of every output and the latest hidden state. Then the decoder is given
the ``<SOS>`` token as its first input, and the last hidden state of the
encoder as its first hidden state.

"Teacher forcing" is the concept of using the real target outputs as
each next input, instead of using the decoder's guess as the next input.
Using teacher forcing causes it to converge faster but `when the trained
network is exploited, it may exhibit
instability <http://minds.jacobs-university.de/sites/default/files/uploads/papers/ESNTutorialRev.pdf>`__.

You can observe outputs of teacher-forced networks that read with
coherent grammar but wander far from the correct translation -
intuitively it has learned to represent the output grammar and can "pick
up" the meaning once the teacher tells it the first few words, but it
has not properly learned how to create the sentence from the translation
in the first place.

Because of the freedom PyTorch's autograd gives us, we can randomly
choose to use teacher forcing or not with a simple if statement. Turn
``teacher_forcing_ratio`` up to use more of it.




In [46]:
teacher_forcing_ratio = 0.5

def train(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion, max_length=MAX_LENGTH):
    encoder_hidden = encoder.initHidden()

    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    input_length = input_tensor.size(0)
    target_length = target_tensor.size(0)

    encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

    loss = 0

    for ei in range(input_length):
        encoder_output, encoder_hidden = encoder(
            input_tensor[ei], encoder_hidden)
        # encoder_outputs[ei] = encoder_output[0, 0]

    decoder_input = torch.tensor([[SOS_token]], device=device)

    decoder_hidden = encoder_hidden

    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

    if use_teacher_forcing:
        # Teacher forcing: Feed the target as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden = decoder(
                decoder_input, decoder_hidden)
            loss += criterion(decoder_output, target_tensor[di])
            decoder_input = target_tensor[di]  # Teacher forcing

    else:
        # Without teacher forcing: use its own predictions as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden = decoder(
                decoder_input, decoder_hidden)
            topv, topi = decoder_output.topk(1)
            decoder_input = topi.squeeze().detach()  # detach from history as input

            loss += criterion(decoder_output, target_tensor[di])
            if decoder_input.item() == EOS_token:
                break

    loss.backward()

    encoder_optimizer.step()
    decoder_optimizer.step()

    return loss.item() / target_length

In [36]:
teacher_forcing_ratio = 0.5

def trainWithAttention(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion, max_length=MAX_LENGTH):
    encoder_hidden = encoder.initHidden()

    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    input_length = input_tensor.size(0)
    target_length = target_tensor.size(0)

    encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

    loss = 0

    for ei in range(input_length):
        encoder_output, encoder_hidden = encoder(
            input_tensor[ei], encoder_hidden)
        encoder_outputs[ei] = encoder_output[0, 0]

    encoder_outputs = encoder_outputs.unsqueeze(0)

    decoder_input = torch.tensor([[SOS_token]], device=device)

    decoder_hidden = encoder_hidden

    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

    if use_teacher_forcing:
        # Teacher forcing: Feed the target as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden, attn_weights = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            loss += criterion(decoder_output, target_tensor[di])
            decoder_input = target_tensor[di]  # Teacher forcing

    else:
        # Without teacher forcing: use its own predictions as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden, attn_weights = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            topv, topi = decoder_output.topk(1)
            decoder_input = topi.squeeze().detach()  # detach from history as input

            loss += criterion(decoder_output, target_tensor[di])
            if decoder_input.item() == EOS_token:
                break

    loss.backward()

    encoder_optimizer.step()
    decoder_optimizer.step()

    return loss.item() / target_length

In [17]:
teacher_forcing_ratio = 0.5

def trainTransformer(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion, max_length=MAX_LENGTH):

    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    input_length = input_tensor.size(0)
    target_length = target_tensor.size(0)

    loss = 0

    encoder_output = encoder(input_tensor)

    encoder_hidden = encoder_output.mean(0).unsqueeze(0)

    decoder_input = torch.tensor([[SOS_token]], device=device)

    decoder_hidden = encoder_hidden

    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

    if use_teacher_forcing:
        # Teacher forcing: Feed the target as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden = decoder(
                decoder_input, decoder_hidden)
            loss += criterion(decoder_output, target_tensor[di])
            decoder_input = target_tensor[di]  # Teacher forcing

    else:
        # Without teacher forcing: use its own predictions as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden = decoder(
                decoder_input, decoder_hidden)
            topv, topi = decoder_output.topk(1)
            decoder_input = topi.squeeze().detach()  # detach from history as input

            loss += criterion(decoder_output, target_tensor[di])
            if decoder_input.item() == EOS_token:
                break

    loss.backward()

    nn.utils.clip_grad_norm_(encoder.parameters(), 0.5)
    nn.utils.clip_grad_norm_(decoder.parameters(), 0.5)

    encoder_optimizer.step()
    decoder_optimizer.step()

    return loss.item() / target_length

This is a helper function to print time elapsed and estimated time
remaining given the current time and progress %.




In [18]:
import time
import math


def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)


def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))

The whole training process looks like this:

-  Start a timer
-  Initialize optimizers and criterion
-  Create set of training pairs
-  Start empty losses array for plotting

Then we call ``train`` many times and occasionally print the progress (%
of examples, time so far, estimated time) and average loss.




In [27]:
def trainIters(encoder, decoder, epochs, print_every=1000, plot_every=100, learning_rate=0.01, attention=False, transformer=False):
    start = time.time()
    plot_losses = []
    print_loss_total = 0  # Reset every print_every
    plot_loss_total = 0  # Reset every plot_every

    encoder_optimizer = optim.SGD(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.SGD(decoder.parameters(), lr=learning_rate)

    criterion = nn.NLLLoss()

    iter = 1
    n_iters = len(train_pairs) * epochs

    for epoch in range(epochs):
        print("Epoch: %d/%d" % (epoch, epochs))
        for training_pair in train_pairs:
            training_pair = tensorsFromPair(training_pair)

            input_tensor = training_pair[0]
            target_tensor = training_pair[1]
            if attention:
                loss = trainWithAttention(input_tensor, target_tensor, encoder,
                        decoder, encoder_optimizer, decoder_optimizer, criterion)
            elif transformer:
                loss = trainTransformer(input_tensor, target_tensor, encoder,
                        decoder, encoder_optimizer, decoder_optimizer, criterion)
            else:
                loss = train(input_tensor, target_tensor, encoder,
                        decoder, encoder_optimizer, decoder_optimizer, criterion)
            print_loss_total += loss
            plot_loss_total += loss

            if iter % print_every == 0:
                print_loss_avg = print_loss_total / print_every
                print_loss_total = 0
                print('%s (%d %d%%) %.4f' % (timeSince(start, iter / n_iters),
                                            iter, iter / n_iters * 100, print_loss_avg))

            iter +=1

Evaluation
==========

Evaluation is mostly the same as training, but there are no targets so
we simply feed the decoder's predictions back to itself for each step.
Every time it predicts a word we add it to the output string, and if it
predicts the EOS token we stop there. We also store the decoder's
attention outputs for display later.




In [21]:
def evaluate(encoder, decoder, sentence, max_length=MAX_LENGTH):
    with torch.no_grad():
        input_tensor = tensorFromSentence(input_lang, sentence)
        input_length = input_tensor.size()[0]
        encoder_hidden = encoder.initHidden()

        encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

        for ei in range(input_length):
            encoder_output, encoder_hidden = encoder(input_tensor[ei],
                                                     encoder_hidden)
            # encoder_outputs[ei] += encoder_output[0, 0]

        decoder_input = torch.tensor([[SOS_token]], device=device)  # SOS

        decoder_hidden = encoder_hidden

        decoded_words = []

        for di in range(max_length):
            decoder_output, decoder_hidden = decoder(
                decoder_input, decoder_hidden)
            topv, topi = decoder_output.data.topk(1)
            if topi.item() == EOS_token:
                decoded_words.append('<EOS>')
                break
            else:
                decoded_words.append(output_lang.index2word[topi.item()])

            decoder_input = topi.squeeze().detach()

        return decoded_words

In [37]:
def evaluateWithAttention(encoder, decoder, sentence, max_length=MAX_LENGTH):
    with torch.no_grad():
        input_tensor = tensorFromSentence(input_lang, sentence)
        input_length = input_tensor.size()[0]
        encoder_hidden = encoder.initHidden()

        encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

        for ei in range(input_length):
            encoder_output, encoder_hidden = encoder(input_tensor[ei],
                                                     encoder_hidden)
            encoder_outputs[ei] += encoder_output[0, 0]

        encoder_outputs = encoder_outputs.unsqueeze(0)

        decoder_input = torch.tensor([[SOS_token]], device=device)  # SOS

        decoder_hidden = encoder_hidden

        decoded_words = []

        for di in range(max_length):
            decoder_output, decoder_hidden, decoder_attn = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            topv, topi = decoder_output.data.topk(1)
            if topi.item() == EOS_token:
                decoded_words.append('<EOS>')
                break
            else:
                decoded_words.append(output_lang.index2word[topi.item()])

            decoder_input = topi.squeeze().detach()

        return decoded_words

In [20]:
def evaluateTransformer(encoder, decoder, sentence, max_length=MAX_LENGTH):
    with torch.no_grad():
        input_tensor = tensorFromSentence(input_lang, sentence)
        input_length = input_tensor.size()[0]

        encoder_output = encoder(input_tensor)

        encoder_hidden = encoder_output.mean(0).unsqueeze(0)

        decoder_input = torch.tensor([[SOS_token]], device=device)  # SOS

        decoder_hidden = encoder_hidden

        decoded_words = []

        for di in range(max_length):
            decoder_output, decoder_hidden = decoder(
                decoder_input, decoder_hidden)
            topv, topi = decoder_output.data.topk(1)
            if topi.item() == EOS_token:
                decoded_words.append('<EOS>')
                break
            else:
                decoded_words.append(output_lang.index2word[topi.item()])

            decoder_input = topi.squeeze().detach()

        return decoded_words

We can evaluate random sentences from the training set and print out the
input, target, and output to make some subjective quality judgements:




In [21]:
def evaluateRandomly(encoder, decoder, n=10, attention=False, transformer=False):
    for i in range(n):
        pair = random.choice(pairs)
        print('>', pair[0])
        print('=', pair[1])
        if attention:
            output_words = evaluateWithAttention(encoder, decoder, pair[0])
        elif transformer:
            output_words = evaluateTransformer(encoder, decoder, pair[0])
        else:
            output_words = evaluate(encoder, decoder, pair[0])
        output_sentence = ' '.join(output_words)
        print('<', output_sentence)
        print('')

In [22]:
from torchmetrics.text.rouge import ROUGEScore
from tqdm import tqdm
import numpy as np

rouge = ROUGEScore()

def test(encoder, decoder, testing_pairs, attention=False, transformer=False):
    input = []
    gt = []
    predict = []
    metric_score = {
        "rouge1_fmeasure":[],
        "rouge1_precision":[],
        "rouge1_recall":[],
        "rouge2_fmeasure":[],
        "rouge2_precision":[],
        "rouge2_recall":[]
    }
    from tqdm import tqdm
    for i in tqdm(range(len(testing_pairs))):
        pair = testing_pairs[i]
        if attention:
            output_words = evaluateWithAttention(encoder, decoder, pair[0])
        elif transformer:
            output_words = evaluateTransformer(encoder, decoder, pair[0])
        else:
            output_words = evaluate(encoder, decoder, pair[0])
        output_sentence = ' '.join(output_words)

        input.append(pair[0])
        gt.append(pair[1])
        predict.append(output_sentence)

        try:
            rs = rouge(output_sentence, pair[1])
        except:
            continue
        metric_score["rouge1_fmeasure"].append(rs['rouge1_fmeasure'])
        metric_score["rouge1_precision"].append(rs['rouge1_precision'])
        metric_score["rouge1_recall"].append(rs['rouge1_recall'])
        metric_score["rouge2_fmeasure"].append(rs['rouge2_fmeasure'])
        metric_score["rouge2_precision"].append(rs['rouge2_precision'])
        metric_score["rouge2_recall"].append(rs['rouge2_recall'])

    metric_score["rouge1_fmeasure"] = np.array(metric_score["rouge1_fmeasure"]).mean()
    metric_score["rouge1_precision"] = np.array(metric_score["rouge1_precision"]).mean()
    metric_score["rouge1_recall"] = np.array(metric_score["rouge1_recall"]).mean()
    metric_score["rouge2_fmeasure"] = np.array(metric_score["rouge2_fmeasure"]).mean()
    metric_score["rouge2_precision"] = np.array(metric_score["rouge2_precision"]).mean()
    metric_score["rouge2_recall"] = np.array(metric_score["rouge2_recall"]).mean()

    print("=== Evaluation score - Rouge score ===")
    print("Rouge1 fmeasure:\t",metric_score["rouge1_fmeasure"])
    print("Rouge1 precision:\t",metric_score["rouge1_precision"])
    print("Rouge1 recall:  \t",metric_score["rouge1_recall"])
    print("Rouge2 fmeasure:\t",metric_score["rouge2_fmeasure"])
    print("Rouge2 precision:\t",metric_score["rouge2_precision"])
    print("Rouge2 recall:  \t",metric_score["rouge2_recall"])
    print("=====================================")
    return input,gt,predict,metric_score

Training and Evaluating
=======================

With all these helper functions in place (it looks like extra work, but
it makes it easier to run multiple experiments) we can actually
initialize a network and start training.

Remember that the input sentences were heavily filtered. For this small
dataset we can use relatively small networks of 256 hidden nodes and a
single GRU layer. After about 40 minutes on a MacBook CPU we'll get some
reasonable results.

.. Note::
   If you run this notebook you can train, interrupt the kernel,
   evaluate, and continue training later. Comment out the lines where the
   encoder and decoder are initialized and run ``trainIters`` again.




In [60]:
hidden_size = 512
encoder1 = EncoderRNN(input_lang.n_words, hidden_size).to(device)
decoder1 = Decoder(hidden_size, output_lang.n_words).to(device)

trainIters(encoder1, decoder1, 2, print_every=5000)

Epoch: 0/2
1m 19s (- 9m 34s) (5000 12%) 3.2489
2m 39s (- 8m 18s) (10000 24%) 2.6442
4m 0s (- 7m 0s) (15000 36%) 2.3568
5m 22s (- 5m 41s) (20000 48%) 2.1502
Epoch: 1/2
6m 43s (- 4m 22s) (25000 60%) 1.9438
8m 5s (- 3m 1s) (30000 72%) 1.7762
9m 28s (- 1m 41s) (35000 84%) 1.5992
10m 51s (- 0m 20s) (40000 97%) 1.5246


In [61]:
evaluateRandomly(encoder1, decoder1)

> je suis heureux d entendre cela .
= i m pleased to hear that .
< i m glad to hear that . <EOS>

> je suis desole je ne vous ai pas reconnues .
= i m sorry i didn t recognize you .
< i m sorry sorry i didn t recognize you . <EOS>

> je suis desole d avoir cause tant de problemes .
= i m sorry i ve caused so many problems .
< i m sorry that i ve got so much . <EOS>

> il est au bout de ses forces .
= he s on his last legs .
< he is an old to on the <EOS>

> tu es imprevisible .
= you re unpredictable .
< you re extroverted . <EOS>

> elle est canadienne .
= she is canadian .
< she s an . <EOS>

> c est un garcon tres intelligent .
= he is a very smart boy .
< he is a very smart boy . <EOS>

> je suis desole d avoir dit cela .
= i m sorry i said that .
< i m sorry that i told . <EOS>

> tu vas tellement me manquer .
= i m going to miss you so much .
< i m going to miss you . <EOS>

> elles ne sont pas dangereuses .
= they re harmless .
< they re not . <EOS>



In [62]:
input,gt,predict,score = test(encoder1, decoder1, train_pairs)

100%|██████████| 20616/20616 [02:48<00:00, 122.67it/s]


=== Evaluation score - Rouge score ===
Rouge1 fmeasure:	 0.6778011
Rouge1 precision:	 0.6392721
Rouge1 recall:  	 0.730681
Rouge2 fmeasure:	 0.5002184
Rouge2 precision:	 0.46313792
Rouge2 recall:  	 0.5539724


In [63]:
input,gt,predict,score = test(encoder1, decoder1, test_pairs)

100%|██████████| 2291/2291 [00:17<00:00, 129.80it/s]


=== Evaluation score - Rouge score ===
Rouge1 fmeasure:	 0.6151996
Rouge1 precision:	 0.58205163
Rouge1 recall:  	 0.66262376
Rouge2 fmeasure:	 0.42305884
Rouge2 precision:	 0.39330986
Rouge2 recall:  	 0.46782106


In [52]:
hidden_size = 512
encoder2 = EncoderLSTM(input_lang.n_words, hidden_size).to(device)
decoder2 = DecoderLSTM(hidden_size, output_lang.n_words).to(device)

trainIters(encoder2, decoder2, 2, print_every=5000)

Epoch: 0/2
1m 25s (- 10m 22s) (5000 12%) 3.3893
2m 52s (- 8m 59s) (10000 24%) 2.9044
4m 19s (- 7m 33s) (15000 36%) 2.6594
5m 47s (- 6m 8s) (20000 48%) 2.4801
Epoch: 1/2
7m 21s (- 4m 46s) (25000 60%) 2.3028
8m 50s (- 3m 18s) (30000 72%) 2.1402
10m 17s (- 1m 50s) (35000 84%) 2.0024
11m 45s (- 0m 21s) (40000 97%) 1.9051


In [53]:
evaluateRandomly(encoder2, decoder2)

> ils sont tous a moi .
= they re all mine .
< they re all mine . <EOS>

> je suis heureux avec ce que j ai .
= i m happy with what i ve got .
< i m happy with what i am . <EOS>

> je suis tres contente de vous revoir .
= i m very glad to see you again .
< i m very happy to you you . <EOS>

> elle fait des progres en anglais .
= she is making progress with her english .
< she is english of english . <EOS>

> je suis desolee si je t ai effraye .
= i m sorry if i frightened you .
< i m sorry if i embarrassed you . <EOS>

> ils vont se marier dans l hotel de ville .
= they are going to get married in the town hall .
< they are going to in in in the . . <EOS>

> je cherche quelqu un .
= i m looking for someone .
< i m looking for a . . <EOS>

> j ai hate de recevoir ta reponse .
= i m looking forward to receiving your reply .
< i m trying to have your your . <EOS>

> je suis desole je ne t ai pas entendue .
= i m sorry i didn t hear you .
< i m sorry sorry i didn t hear you . <EOS>

> je s

In [54]:
input,gt,predict,score = test(encoder2, decoder2, train_pairs)

100%|██████████| 20616/20616 [02:39<00:00, 129.29it/s]


=== Evaluation score - Rouge score ===
Rouge1 fmeasure:	 0.60194266
Rouge1 precision:	 0.5824576
Rouge1 recall:  	 0.63163066
Rouge2 fmeasure:	 0.4020284
Rouge2 precision:	 0.38353485
Rouge2 recall:  	 0.43136868


In [55]:
input,gt,predict,score = test(encoder2, decoder2, test_pairs)

100%|██████████| 2291/2291 [00:18<00:00, 126.86it/s]


=== Evaluation score - Rouge score ===
Rouge1 fmeasure:	 0.5743263
Rouge1 precision:	 0.55588657
Rouge1 recall:  	 0.603229
Rouge2 fmeasure:	 0.3737726
Rouge2 precision:	 0.3568855
Rouge2 recall:  	 0.40104398


In [90]:
hidden_size = 512
encoder3 = EncoderLSTM(input_lang.n_words, hidden_size, True).to(device)
decoder3 = Decoder(hidden_size*2, output_lang.n_words).to(device)

trainIters(encoder3, decoder3, 2, print_every=5000)

Epoch: 0/2
2m 12s (- 16m 0s) (5000 12%) 3.3569
4m 27s (- 13m 56s) (10000 24%) 2.7412
6m 44s (- 11m 46s) (15000 36%) 2.4254
9m 2s (- 9m 35s) (20000 48%) 2.2189
Epoch: 1/2
11m 21s (- 7m 22s) (25000 60%) 2.0024
13m 40s (- 5m 7s) (30000 72%) 1.8345
15m 59s (- 2m 50s) (35000 84%) 1.6598
18m 18s (- 0m 33s) (40000 97%) 1.5647


In [91]:
evaluateRandomly(encoder3, decoder3)

> nous sommes prets a prendre ce risque .
= we re prepared to take that risk .
< we re ready to take that risk . <EOS>

> nous te demandons de ne pas faire ca .
= we re asking you not to do that .
< you re not asking to do that . <EOS>

> je suis bourre mais je sais encore parler allemand .
= i m drunk but i can still speak german .
< i m still but i ll speak . <EOS>

> j essaye d etre heureux .
= i m trying to be happy .
< i m trying to be happy . <EOS>

> elle passe chaque dimanche du temps avec sa grand mere .
= she spends time with her grandmother every sunday .
< she spends a lot of time time time time . <EOS>

> vous n etes pas vraiment millionnaire si ?
= you aren t really a millionaire are you ?
< you really are not a millionaire are you ? <EOS>

> il est en train de prendre son dejeuner a l heure actuelle .
= he s eating lunch now .
< he is always time time time . <EOS>

> je suis sur .
= i m certain .
< i m on . <EOS>

> je vais garder mon emploi .
= i m going to keep my job 

In [92]:
input,gt,predict,score = test(encoder3, decoder3, train_pairs)

100%|██████████| 20616/20616 [03:03<00:00, 112.57it/s]


=== Evaluation score - Rouge score ===
Rouge1 fmeasure:	 0.67177624
Rouge1 precision:	 0.64845437
Rouge1 recall:  	 0.7074254
Rouge2 fmeasure:	 0.4869261
Rouge2 precision:	 0.46232003
Rouge2 recall:  	 0.5254809


In [93]:
input,gt,predict,score = test(encoder3, decoder3, test_pairs)

100%|██████████| 2291/2291 [00:20<00:00, 111.99it/s]


=== Evaluation score - Rouge score ===
Rouge1 fmeasure:	 0.6156294
Rouge1 precision:	 0.5960008
Rouge1 recall:  	 0.64722306
Rouge2 fmeasure:	 0.41964275
Rouge2 precision:	 0.39997742
Rouge2 recall:  	 0.45177808


In [38]:
hidden_size = 512
encoder4 = EncoderRNN(input_lang.n_words, hidden_size).to(device)
decoder4 = AttentionDecoder(hidden_size, output_lang.n_words).to(device)

trainIters(encoder4, decoder4, 2, print_every=5000, attention=True)

Epoch: 0/2
1m 46s (- 12m 50s) (5000 12%) 3.1946
3m 32s (- 11m 4s) (10000 24%) 2.5486
5m 20s (- 9m 20s) (15000 36%) 2.2714
7m 9s (- 7m 35s) (20000 48%) 2.0966
Epoch: 1/2
8m 57s (- 5m 48s) (25000 60%) 1.9108
10m 44s (- 4m 1s) (30000 72%) 1.7317
12m 33s (- 2m 14s) (35000 84%) 1.5691
14m 22s (- 0m 26s) (40000 97%) 1.5006


In [39]:
evaluateRandomly(encoder4, decoder4, attention=True)

> je ne te vends pas ma voiture .
= i m not selling you my car .
< i m not selling you my car . . <EOS>

> il faut que j y aille .
= i must go .
< i must be going . <EOS>

> elle semble etre impliquee dans cette affaire de meurtre .
= she seems to be involved in that murder case .
< she seems supposed to be in with that . <EOS>

> je me promene dans un parc .
= i m taking a walk in a park .
< i m a a a a a a a a a a <EOS>

> je suis desolee ma prononciation n est pas tres bonne .
= i m sorry my pronunciation isn t very good .
< i m sorry my is not good . . . <EOS>

> nous nous amusons .
= we re having fun .
< we re dependent on . <EOS>

> tu es une celebrite .
= you re a star .
< you re a dreamer . <EOS>

> nous ne faisons que commencer .
= we re just beginning .
< we re just just . . <EOS>

> tu es la seule canadienne que je connaisse .
= you re the only canadian i know .
< you re the only person i know know . <EOS>

> tu es vraiment egoiste .
= you re really selfish .
< you re really

In [40]:
input,gt,predict,score = test(encoder4, decoder4, train_pairs, attention=True)

100%|██████████| 20616/20616 [03:33<00:00, 96.47it/s] 


=== Evaluation score - Rouge score ===
Rouge1 fmeasure:	 0.6767843
Rouge1 precision:	 0.6351408
Rouge1 recall:  	 0.73492646
Rouge2 fmeasure:	 0.501975
Rouge2 precision:	 0.4626029
Rouge2 recall:  	 0.5599345


In [41]:
input,gt,predict,score = test(encoder4, decoder4, test_pairs, attention=True)

100%|██████████| 2291/2291 [00:23<00:00, 95.93it/s] 


=== Evaluation score - Rouge score ===
Rouge1 fmeasure:	 0.6167475
Rouge1 precision:	 0.5807893
Rouge1 recall:  	 0.6691904
Rouge2 fmeasure:	 0.42953944
Rouge2 precision:	 0.39835694
Rouge2 recall:  	 0.47729892


In [23]:
hidden_size = 512
nhead = 8 # number of heads in ``nn.MultiheadAttention``
num_layers = 1 # number of ``nn.TransformerEncoderLayer`` in ``nn.TransformerEncoder``
encoder5 = EncoderTransformer(input_lang.n_words, hidden_size, nhead, num_layers).to(device)
decoder5 = Decoder(hidden_size, output_lang.n_words).to(device)

trainIters(encoder5, decoder5, 8, print_every=5000, learning_rate=1, transformer=True)

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/transformer.py:286: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


Epoch: 0/8
1m 0s (- 32m 5s) (5000 3%) 3.2517
1m 58s (- 30m 33s) (10000 6%) 2.8916
2m 57s (- 29m 34s) (15000 9%) 2.7616
3m 56s (- 28m 35s) (20000 12%) 2.6673
Epoch: 1/8
4m 55s (- 27m 36s) (25000 15%) 2.5343
5m 55s (- 26m 36s) (30000 18%) 2.4382
6m 54s (- 25m 37s) (35000 21%) 2.3760
7m 53s (- 24m 37s) (40000 24%) 2.3467
Epoch: 2/8
8m 52s (- 23m 38s) (45000 27%) 2.2471
9m 51s (- 22m 40s) (50000 30%) 2.1710
10m 51s (- 21m 41s) (55000 33%) 2.1193
11m 50s (- 20m 43s) (60000 36%) 2.1058
Epoch: 3/8
12m 50s (- 19m 44s) (65000 39%) 2.0405
13m 49s (- 18m 45s) (70000 42%) 1.9832
14m 49s (- 17m 46s) (75000 45%) 1.9244
15m 49s (- 16m 47s) (80000 48%) 1.8978
Epoch: 4/8
16m 48s (- 15m 48s) (85000 51%) 1.8676
17m 48s (- 14m 49s) (90000 54%) 1.8166
18m 48s (- 13m 50s) (95000 57%) 1.7580
19m 47s (- 12m 51s) (100000 60%) 1.7337
Epoch: 5/8
20m 46s (- 11m 51s) (105000 63%) 1.7360
21m 46s (- 10m 52s) (110000 66%) 1.6804
22m 45s (- 9m 52s) (115000 69%) 1.6498
23m 44s (- 8m 53s) (120000 72%) 1.6506
Epoch: 6/8


In [24]:
evaluateRandomly(encoder5, decoder5, transformer=True)

> on y va en train .
= we re going by train .
< we re going to eat this . <EOS>

> je suis un vendeur .
= i m a salesman .
< i m a salesperson . <EOS>

> il est bon cuisinier .
= he is good at cooking .
< he is good at cards . <EOS>

> il est avide de gloire .
= he is eager to become famous .
< he is eager . <EOS>

> il n est pas assez age pour voter .
= he is not old enough to vote .
< he is not old enough to vote . <EOS>

> nous sommes fiers de nos enfants .
= we re proud of our children .
< we re proud our our our . . <EOS>

> nous avons un week end de trois jours cette semaine .
= we re having a three day weekend this week .
< he is a a deal of this kind of <EOS>

> tu es libre .
= you re free .
< you re free . <EOS>

> nous jouons a un jeu .
= we re playing a game .
< we received a large game . <EOS>

> il me faut me cacher .
= i must hide .
< i must insist . <EOS>



In [25]:
input,gt,predict,score = test(encoder5, decoder5, train_pairs, transformer=True)

100%|██████████| 20616/20616 [02:13<00:00, 154.51it/s]


=== Evaluation score - Rouge score ===
Rouge1 fmeasure:	 0.6069931
Rouge1 precision:	 0.5752796
Rouge1 recall:  	 0.65336096
Rouge2 fmeasure:	 0.42384988
Rouge2 precision:	 0.39239824
Rouge2 recall:  	 0.4711976


In [26]:
input,gt,predict,score = test(encoder5, decoder5, test_pairs, transformer=True)

100%|██████████| 2291/2291 [00:14<00:00, 155.56it/s]


=== Evaluation score - Rouge score ===
Rouge1 fmeasure:	 0.55557185
Rouge1 precision:	 0.528901
Rouge1 recall:  	 0.5966636
Rouge2 fmeasure:	 0.36217645
Rouge2 precision:	 0.33733237
Rouge2 recall:  	 0.40123042
